In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import package
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Input, Dense, Dropout, Activation,
                                     BatchNormalization, Flatten,
                                     Conv2D, MaxPooling2D)

In [ ]:
tf.__version__

'2.3.0'

In [ ]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('x_train.shape:', x_train.shape)
print('y_train.shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 12s 0us/step
x_train.shape: (50000, 32, 32, 3)
y_train.shape: (50000, 1)
50000 train samples
10000 test samples


In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255  # rescaling
x_test /= 255   # rescaling

In [ ]:
# 將訓練資料與測試資料的 label，進行 Onehot encoding 轉換
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print('y_test.argmax(1) shape:', y_test.argmax(1).shape)

y_train shape: (50000, 10)
y_test shape: (10000, 10)
y_test.argmax(1) shape: (10000,)


In [ ]:
#確認CPU&GPU裝置狀況
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15376152850213874633
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 820837139109308112
physical_device_desc: "device: XLA_CPU device"
]


In [ ]:
from keras.models import load_model
VGG_model = load_model('/content/drive/MyDrive/Colab Notebooks/edge_AI/VGG/VGG_model_org.h5')
NASNetLarge_model = load_model('/content/drive/MyDrive/Colab Notebooks/edge_AI/NASNetLarge2/NASNetLarge2_model_org.h5')
MobileNetV2_model = load_model('/content/drive/MyDrive/Colab Notebooks/edge_AI/MobileNetV2/MobileNetV2_model_org.h5')

## Post training Quantization 模型最佳化
[參考官網 - Post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization)

In [ ]:
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
from tensorflow import keras
import numpy as np
import pathlib

In [ ]:
#使用TFLiteConverter來載入模型
VGG_converter = tf.lite.TFLiteConverter.from_keras_model(VGG_model)
NASNetLarge_converter = tf.lite.TFLiteConverter.from_keras_model(NASNetLarge_model)
MobileNetV2_converter = tf.lite.TFLiteConverter.from_keras_model(MobileNetV2_model)
#執行模型格式轉換 -> 轉成TFLite格式
VGG_tflite_model = VGG_converter.convert()
NASNetLarge_tflite_model = NASNetLarge_converter.convert()
MobileNetV2_tflite_model = MobileNetV2_converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tmp5x7vnd66/assets
INFO:tensorflow:Assets written to: /tmp/tmpovoq4n26/assets


INFO:tensorflow:Assets written to: /tmp/tmpovoq4n26/assets


INFO:tensorflow:Assets written to: /tmp/tmprqyhg92k/assets


INFO:tensorflow:Assets written to: /tmp/tmprqyhg92k/assets


In [ ]:
VGG_tflite_file = '/content/drive/MyDrive/Colab Notebooks/edge_AI/VGG/VGG_model.tflite'
NASNetLarge_tflite_file = '/content/drive/MyDrive/Colab Notebooks/edge_AI/NASNetLarge2/NASNetLarge2_model.tflite'
MobileNetV2_tflite_file = '/content/drive/MyDrive/Colab Notebooks/edge_AI/MobileNetV2/MobileNetV2_model.tflite'

with open(VGG_tflite_file, 'wb') as f:
    f.write(VGG_tflite_model)

print('Saved VGG TFLite model to:', VGG_tflite_file)

with open(NASNetLarge_tflite_file, 'wb') as f:
    f.write(NASNetLarge_tflite_model)

print('Saved NASNetLarge TFLite model to:', NASNetLarge_tflite_file)

with open(MobileNetV2_tflite_file, 'wb') as f:
    f.write(MobileNetV2_tflite_model)

print('Saved MobileNetV2 TFLite model to:', MobileNetV2_tflite_file)

Saved VGG TFLite model to: /content/drive/MyDrive/Colab Notebooks/edge_AI/VGG/VGG_model.tflite
Saved NASNetLarge TFLite model to: /content/drive/MyDrive/Colab Notebooks/edge_AI/NASNetLarge2/NASNetLarge2_model.tflite
Saved MobileNetV2 TFLite model to: /content/drive/MyDrive/Colab Notebooks/edge_AI/MobileNetV2/MobileNetV2_model.tflite


In [ ]:
#設定最佳化參數
VGG_converter.optimizations = [tf.lite.Optimize.DEFAULT]
NASNetLarge_converter.optimizations = [tf.lite.Optimize.DEFAULT]
MobileNetV2_converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.optimizations.optimizations = [tf.lite.Optimize.DEFAULT]

#執行最佳化轉換
VGG_tflite_quant_model = VGG_converter.convert()
NASNetLarge_tflite_quant_model = NASNetLarge_converter.convert()
MobileNetV2_tflite_quant_model = MobileNetV2_converter.convert()

VGG_quanted_tflite_file = '/content/drive/MyDrive/Colab Notebooks/edge_AI/VGG/VGG_model_quant.tflite'
NASNetLarge_quanted_tflite_file = '/content/drive/MyDrive/Colab Notebooks/edge_AI/NASNetLarge2/NASNetLarge2_model_quant.tflite'
MobileNetV2_quanted_tflite_file = '/content/drive/MyDrive/Colab Notebooks/edge_AI/MobileNetV2/MobileNetV2_model_quant.tflite'

with open(VGG_quanted_tflite_file, 'wb') as f:
    f.write(VGG_tflite_quant_model)

print('Saved VGG TFLite model to:', VGG_quanted_tflite_file)

with open(NASNetLarge_quanted_tflite_file, 'wb') as f:
    f.write(NASNetLarge_tflite_quant_model)

print('Saved NASNetLarge TFLite model to:', NASNetLarge_quanted_tflite_file)

with open(MobileNetV2_quanted_tflite_file, 'wb') as f:
    f.write(MobileNetV2_tflite_quant_model)

print('Saved MobileNetV2 TFLite model to:', MobileNetV2_quanted_tflite_file)

INFO:tensorflow:Assets written to: /tmp/tmp5asvpenc/assets


INFO:tensorflow:Assets written to: /tmp/tmp5asvpenc/assets


INFO:tensorflow:Assets written to: /tmp/tmp3uiiy62h/assets


INFO:tensorflow:Assets written to: /tmp/tmp3uiiy62h/assets


INFO:tensorflow:Assets written to: /tmp/tmpl7w8nomh/assets


INFO:tensorflow:Assets written to: /tmp/tmpl7w8nomh/assets


Saved VGG TFLite model to: /content/drive/MyDrive/Colab Notebooks/edge_AI/VGG/VGG_model_quant.tflite
Saved NASNetLarge TFLite model to: /content/drive/MyDrive/Colab Notebooks/edge_AI/NASNetLarge2/NASNetLarge2_model_quant.tflite
Saved MobileNetV2 TFLite model to: /content/drive/MyDrive/Colab Notebooks/edge_AI/MobileNetV2/MobileNetV2_model_quant.tflite


## 執行 TFLite models (使用解譯器 interpreter)

In [ ]:
#載入原本的TFLite model
VGG_interpreter = tf.lite.Interpreter(model_path=str(VGG_tflite_file))
NASNetLarge_interpreter = tf.lite.Interpreter(model_path=str(NASNetLarge_tflite_file))
MobileNetV2_interpreter = tf.lite.Interpreter(model_path=str(MobileNetV2_tflite_file))

VGG_interpreter.allocate_tensors()
NASNetLarge_interpreter.allocate_tensors()
MobileNetV2_interpreter.allocate_tensors()

In [ ]:
#載入最佳化(quant)後的TFLite model
VGG_interpreter_quant = tf.lite.Interpreter(model_path=str(VGG_quanted_tflite_file))
NASNetLarge_interpreter_quant = tf.lite.Interpreter(model_path=str(NASNetLarge_quanted_tflite_file))
MobileNetV2_interpreter_quant = tf.lite.Interpreter(model_path=str(MobileNetV2_quanted_tflite_file))

VGG_interpreter_quant.allocate_tensors()
NASNetLarge_interpreter_quant.allocate_tensors()
MobileNetV2_interpreter_quant.allocate_tensors()

In [ ]:
#查看input資訊
VGG_interpreter.get_input_details()
NASNetLarge_interpreter.get_input_details()
MobileNetV2_interpreter.get_input_details()

[{'dtype': numpy.float32,
  'index': 0,
  'name': 'input_14',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([ 1, 32, 32,  3], dtype=int32),
  'shape_signature': array([-1, 32, 32,  3], dtype=int32),
  'sparsity_parameters': {}}]

In [ ]:
#查看output資訊
VGG_interpreter.get_output_details()
NASNetLarge_interpreter.get_output_details()
MobileNetV2_interpreter.get_output_details()

[{'dtype': numpy.float32,
  'index': 190,
  'name': 'Identity',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([ 1, 10], dtype=int32),
  'shape_signature': array([-1, 10], dtype=int32),
  'sparsity_parameters': {}}]

## 評估模型準確率損失狀況

In [ ]:
# source by: Copyright 2019 The TensorFlow Authors
# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model(interpreter, test_images, test_labels):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    # note: the "digit" is image class id
    # Run predictions on every image in the "test" dataset.
    prediction_digits = []
    for test_image in test_images:
        # Pre-processing: add batch dimension and convert to float32 to match with
        # the model's input data format.
        test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_image)

        # Run inference.
        interpreter.invoke()

        # Post-processing: remove batch dimension and find the digit with highest
        # probability.
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])
        prediction_digits.append(digit)

    # Compare prediction results with ground truth labels to calculate accuracy.
    accurate_count = 0
    for index in range(len(prediction_digits)):
        if prediction_digits[index] == test_labels[index]:
            accurate_count += 1
    accuracy = accurate_count * 1.0 / len(prediction_digits)

    return accuracy

In [ ]:
print(evaluate_model(VGG_interpreter, x_test, y_test.argmax(1)))
print(evaluate_model(NASNetLarge_interpreter, x_test, y_test.argmax(1)))
print(evaluate_model(MobileNetV2_interpreter, x_test, y_test.argmax(1)))

0.6188
0.4377
0.5959


In [ ]:
print(evaluate_model(VGG_interpreter_quant, x_test, y_test.argmax(1)))
print(evaluate_model(NASNetLarge_interpreter_quant, x_test, y_test.argmax(1)))
print(evaluate_model(MobileNetV2_interpreter_quant, x_test, y_test.argmax(1)))

0.6183
0.4376


---